In [2]:
# Import libraries
import random
import requests
import json

In [3]:
def decimal_to_binary(value, num_bits=8):
    """
    Convert a decimal integer value to its binary representation with a specified number of bits.

    args:
        value (int): The decimal value to be converted to binary. Ex. 10
        num_bits (int): The number of bits in the resulting binary number. Default value is 8.

    Returns:
        str: The binary representation of the decimal value, padded with zeroes to the specified number of bits. Ex. '00001010'
    """
    # Convert the decimal value to binary and remove the '0b' prefix
    binary_value = bin(value)[2:]

    # Pad the binary value with zeroes to the specified number of bits
    padded_binary_value = binary_value.zfill(num_bits)

    return padded_binary_value

In [4]:
def binary_to_decimal(value):
    """
    Convert a binary value to its decimal representation.

    args:
        value (str): The binary value to be converted to decimal. Ex. '00001010'

    Returns:
        int: The decimal representation of the binary value. Ex. 10
    """
    return int(value, 2)

In [5]:
def generate_chromosome(chromosome_size):
    """
    Generate a chromosome consisting of genes with random values from 0 to 255.

    args:
        chromosome_size (int): The number of genes in the chromosome. Ex. 3

    Returns:
        list: A list of random integers from 0 to 255. Ex. [10, 20, 30]
    """
    return [random.randint(0, 255) for _ in range(chromosome_size)]

In [6]:
def generate_population(population_size, chromosome_size):
    """
    Generate a population of chromosomes of a given size.

    args:
        population_size (int): The number of inividuals in the population.
        chromosome_size (int): The number of chromosomes in each individual.

    Returns:
        list: A list of inividuals, each consisting of chromosomes with random values from 0 to 255.
    """
    return [generate_chromosome(chromosome_size) for _ in range(population_size)]

In [7]:
def crossover(best, population):
    """
    Perform one-point crossover between the best indivual and a population of individuals.
    To do this, the genes are converted to binary, and a random crossover point is selected.
    """

    # Convert the genes to binary
    best_binary = [decimal_to_binary(gene) for gene in best]
    population_binary = [
        [decimal_to_binary(gene) for gene in individual] for individual in population
    ]

    # Perform one-point crossover for each gene in the chromosome
    crossover_population = []
    for individual in population_binary:
        crossover_individual = []
        for best_gene, individual_gene in zip(best_binary, individual):
            # Select a random crossover point
            crossover_point = random.randint(0, len(best_gene))
            # Perform crossover
            crossover_gene = (
                best_gene[:crossover_point] + individual_gene[crossover_point:]
            )
            crossover_individual.append(crossover_gene)
            # print(f"Best: {best_gene} | Random: {individual_gene} | Crossover: {crossover_point} | Crossover: {crossover_gene}")
        crossover_population.append(crossover_individual)

    # Convert the binary genes back to decimal
    crossover_population = [
        [binary_to_decimal(gene) for gene in individual]
        for individual in crossover_population
    ]
    # print(f"Crossover: {crossover_population}")

    return crossover_population

In [8]:
def mutation(population, mutation_rate=0.05):
    """
    Perform mutation on a population of individuals.
    To do this, the genes are converted to binary, each bit is flipped with a probability of mutation_rate, and the genes are converted back to decimal.
    """

    # Convert the genes to binary
    population_binary = [
        [decimal_to_binary(gene) for gene in individual] for individual in population
    ]
    # print(population_binary)

    # Perform mutation for each gene in the chromosome, flipping each bit with a probability of mutation_rate
    mutated_population = []
    for individual in population_binary:
        mutated_individual = []
        for gene in individual:
            mutated_gene = "".join(
                [
                    str(int(bit) ^ 1) if random.random() < mutation_rate else bit
                    for bit in gene
                ]
            )
            mutated_individual.append(mutated_gene)
            # print(f"Original: {gene} | Mutated: {mutated_gene}")
        mutated_population.append(mutated_individual)

    # Convert the binary genes back to decimal
    mutated_population = [
        [binary_to_decimal(gene) for gene in individual]
        for individual in mutated_population
    ]
    # print(f"Mutated: {mutated_population}")

    return mutated_population

In [9]:
def generate_new_population(population, fitness_values):
    # Select the best individual
    best_index = fitness_values.index(max(fitness_values))
    best_individual = population[best_index]
    print(f"Population: {population}")
    print(f"Fitness: {fitness_values}")
    print(f"Best: {best_individual}")

    # Generate a random population
    population = generate_population(30, 3)
    print(f"Int Population: {population}")

    # Perform crossover between the best individual and each individual in the population
    crossovered_population = crossover(best_individual, population)
    print(f"Crossover: {crossovered_population}")

    # Mutate the crossovered population with a probability of 0.05
    mutated_population = mutation(crossovered_population)
    print(f"Mutated: {mutated_population}")

    # Select 6 individuals from the mutated population
    new_population = random.sample(mutated_population, 6)

    # Replace the best individual in the same position in the new population
    new_population[best_index] = best_individual
    print(f"New Population: {new_population}")

    return new_population

In [10]:
def update_entities(population):

    URL_LIGHT_FIXTURE_1 = "http://10.24.1.10:1026/v2/entities/urn:ngsi-ld:light_fixture:001/attrs?type=light_fixture"
    URL_LIGHT_FIXTURE_2 = "http://10.24.1.10:1026/v2/entities/urn:ngsi-ld:light_fixture:002/attrs?type=light_fixture"
    URL_LIGHT_FIXTURE_3 = "http://10.24.1.10:1026/v2/entities/urn:ngsi-ld:light_fixture:003/attrs?type=light_fixture"

    HEADERS = {
        "Content-Type": "application/json",
        "fiware-service": "vfarm",
        "fiware-servicepath": "/vfarm",
    }

    # Create payload for each light fixture
    payload_1l = json.dumps(
        {"setLeftColor": {"type": "command", "value": population[0]}}
    )
    payload_1r = json.dumps(
        {"setRightColor": {"type": "command", "value": population[1]}}
    )
    payload_2l = json.dumps(
        {"setLeftColor": {"type": "command", "value": population[2]}}
    )
    payload_2r = json.dumps(
        {"setRightColor": {"type": "command", "value": population[3]}}
    )
    payload_3l = json.dumps(
        {"setLeftColor": {"type": "command", "value": population[4]}}
    )
    payload_3r = json.dumps(
        {"setRightColor": {"type": "command", "value": population[5]}}
    )

    response = requests.request(
        "PATCH", URL_LIGHT_FIXTURE_1, headers=HEADERS, data=payload_1l
    )
    print(response.text)
    response = requests.request(
        "PATCH", URL_LIGHT_FIXTURE_1, headers=HEADERS, data=payload_1r
    )
    print(response.text)
    response = requests.request(
        "PATCH", URL_LIGHT_FIXTURE_2, headers=HEADERS, data=payload_2l
    )
    print(response.text)
    response = requests.request(
        "PATCH", URL_LIGHT_FIXTURE_2, headers=HEADERS, data=payload_2r
    )
    print(response.text)
    response = requests.request(
        "PATCH", URL_LIGHT_FIXTURE_3, headers=HEADERS, data=payload_3l
    )
    print(response.text)
    response = requests.request(
        "PATCH", URL_LIGHT_FIXTURE_3, headers=HEADERS, data=payload_3r
    )
    print(response.text)